<a href="https://colab.research.google.com/github/elianderlohr/muse-dlf/blob/feature%2Fexplainability/notebooks/explainablity/slmuse-dlf-explainability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SLMuSE-DLF Explainability

Plot the explainability of the SLMuSE-DLF model. By using the dictionary learning approach it is (1.) possible to extract how different words in a certaim semantic role predict the presence of a document level frame and (2.) identify how the FrameAxis constallations are predicting the document level frames.

In [1]:
# auto reload imports
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls /content/drive/MyDrive/Git/muse-dlf

assets	data  notebooks  README.md  research-notebooks	run  src  tests  ToDo.md


In [4]:
!pip install wandb allennlp allennlp-models spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.2/730.2 kB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.5/464.5 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.2/248.2 kB 33.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 13.8 MB/s eta 0:00:00
   

In [5]:
!python -m spacy download en_core_web_sm

2024-07-11 19:13:24.944184: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-11 19:13:24.944248: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-11 19:13:25.062324: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-11 19:13:25.259839: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-11 19:13:28.744251: W tensorflow/comp

In [6]:
import sys
sys.path.append('/content/drive/MyDrive/Git/muse-dlf/src')

In [21]:
from preprocessing.pre_processor import PreProcessor
from preprocessing.datasets.article_dataset import custom_collate_fn
from model.slmuse_dlf.muse import SLMUSEDLF

# import tokenizer for roberta fast
from transformers import RobertaTokenizerFast
import wandb
import inspect
import torch
import spacy
import pickle
from pathlib import Path
from torch.utils.data import DataLoader

In [8]:
import nltk

In [9]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [11]:
base_path = "drive/MyDrive/Git/"

## Setup wandb

In [12]:
run = wandb.init(project="slmuse-dlf")

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: ERROR API key must be 40 characters long, yours was 435


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## Load SLMuSE-DLF

In [13]:
model_artifact = run.use_artifact('elianderlohr/slmuse-dlf/model:v12', type='model')
model_dir = model_artifact.download()

wandb: Downloading large artifact model:v12, 491.80MB. 1 files... Done. 0:0:19.5


## Load Roberta Model

In [14]:
roberta_artifact = run.use_artifact('elianderlohr-org/wandb-registry-model/mfc-roberta-finetune:v1', type='model')
roberta_dir = roberta_artifact.download()

wandb: Downloading large artifact mfc-roberta-finetune:v1, 1427.32MB. 7 files... Done. 0:0:40.8


## Load Dataset

In [15]:
dataset_artifact = run.use_artifact('elianderlohr-org/wandb-registry-dataset/slmuse-dlf:v3', type='dataset')
dataset_dir = dataset_artifact.download()

## Load Config

In [17]:
# Access the run that created the artifactelianderlohr/slmuse-dlf/qa9dh6px
run_id = 'elianderlohr/slmuse-dlf/7f8vqzv0'  # Replace with your run ID if known, otherwise see below for how to get it
run_ref = wandb.Api().run(run_id)

In [18]:
# Access the configuration
config = run_ref.config

In [22]:
# Get the parameters of the SLMUSEDLF class constructor
params = inspect.signature(SLMUSEDLF.__init__).parameters

# Extract the relevant parameters from the config dictionary
model_params = {key: config[key] for key in params if key in config}

In [23]:
model_params["bert_model_name"] = "roberta-base"
model_params["bert_model_name_or_path"] = roberta_dir

## Load Model

In [24]:
model = SLMUSEDLF(**model_params)

Some weights of the model checkpoint at ./artifacts/roberta-base-finetune-checkpoint-16482:v0 were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ./artifacts/roberta-base-finetune-checkpoint-16482:v0 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should 

In [25]:
def strip_prefix_from_state_dict(state_dict, prefix):
    """Strip a prefix from the keys in state_dict."""
    return {key[len(prefix):]: value for key, value in state_dict.items() if key.startswith(prefix)}

# Assuming you load the state_dict as follows
state_dict = torch.load(f"{model_dir}/model.pth", map_location="cpu")

In [28]:
stripped_state_dict = strip_prefix_from_state_dict(state_dict, 'module.module.')
model_state_dict = model.state_dict()
model_state_dict.update(stripped_state_dict)

In [29]:
model.load_state_dict(model_state_dict)

RuntimeError: Error(s) in loading state_dict for SLMUSEDLF:
	size mismatch for supervised.feed_forward_sentence.0.weight: copying a param with shape torch.Size([768, 768]) from checkpoint, the shape in current model is torch.Size([24576, 24576]).
	size mismatch for supervised.feed_forward_sentence.0.bias: copying a param with shape torch.Size([768]) from checkpoint, the shape in current model is torch.Size([24576]).
	size mismatch for supervised.feed_forward_sentence.1.weight: copying a param with shape torch.Size([768]) from checkpoint, the shape in current model is torch.Size([24576]).
	size mismatch for supervised.feed_forward_sentence.1.bias: copying a param with shape torch.Size([768]) from checkpoint, the shape in current model is torch.Size([24576]).
	size mismatch for supervised.feed_forward_sentence.1.running_mean: copying a param with shape torch.Size([768]) from checkpoint, the shape in current model is torch.Size([24576]).
	size mismatch for supervised.feed_forward_sentence

## Create Dataset

In [ ]:
!ls drive/MyDrive/Git/muse-dlf/data/axis

In [ ]:
class_column_names = "Capacity and Resources;Crime and Punishment;Cultural Identity;Economic;External Regulation and Reputation;Fairness and Equality;Health and Safety;Legality, Constitutionality, Jurisdiction;Morality;Other;Policy Prescription and Evaluation;Political;Public Sentiment;Quality of Life;Security and Defense".split(";")

### Create Full Dataset

In [ ]:
# Define paths to the dataset files within the downloaded directory
train_artifact_filepath = Path(dataset_dir) / 'train_dataset_artifact.pkl'
test_artifact_filepath = Path(dataset_dir) / 'test_dataset_artifact.pkl'

# Load the datasets from the artifact files
with train_artifact_filepath.open("rb") as f:
    loaded_train_dataset = pickle.load(f)

with test_artifact_filepath.open("rb") as f:
    loaded_test_dataset = pickle.load(f)

In [ ]:
# create dataloaders
train_dataloader = DataLoader(
    loaded_train_dataset,
    batch_size=config["batch_size"],
    shuffle=True,
    collate_fn=custom_collate_fn,
    drop_last=True,
    pin_memory=True,
    num_workers=1,
)

test_dataloader = DataLoader(
    loaded_test_dataset,
    batch_size=config["batch_size"],
    shuffle=True,
    collate_fn=custom_collate_fn,
    drop_last=True,
    pin_memory=True,
    num_workers=1,
)

### Create Example Dataset

In [ ]:
preprocessor = PreProcessor(
    tokenizer=tokenizer,
    batch_size=config["batch_size"],
    max_sentences_per_article=config["num_sentences"],
    max_sentence_length=config["max_sentence_length"],
    max_args_per_sentence=config["max_args_per_sentence"],
    max_arg_length=config["max_arg_length"],
    frameaxis_dim=config["frameaxis_dim"],
    bert_model_name="roberta-base",
    name_tokenizer="roberta-base",
    path_name_bert_model=roberta_dir,
    path_antonym_pairs=f"{base_path}muse-dlf/data/axis/mft.json",
    dim_names=["virtue", "vice"],
    class_column_names=class_column_names,
    )

In [ ]:
text = "BILL ON IMMIGRANT WORKERS DIES. Legislation to allow nearly twice as many computer-savvy foreigners and other high-skilled immigrants into the country next year apparently has died in Congress. The House passed the compromise measure last month, 288-133, but Sen. Tom Harkin, D-Iowa, had blocked a vote when in the Senate. The proposal, backed by high-tech companies, would raise the limit of so- called H-1B visas granted each year to skilled workers from abroad. Only 65,000 visas are now granted each year; the bill would raise the annual cap to 115,500 for the next two years and to 107,500 in 2001. The ceiling would return to 65,000 in 2002."

In [ ]:
example_dataset, example_dataloader = preprocessor.preprocess_single_article(
    text
)

## Run model with data

In [ ]:
from tqdm.notebook import tqdm
import numpy as np
import torch

def inspect(model, dataloader, device='cuda'):
    """
    Make predictions with the given model and dataloader.

    Args:
    - model (torch.nn.Module): The model to make predictions with.
    - dataloader (DataLoader): DataLoader for the dataset to predict on.
    - device (str): Device to make predictions on ('cpu' or 'cuda').

    Returns:
    - predicted_labels (list of lists): List containing the predicted labels for each instance.
    """
    model.eval()

    # dim
    batch_size = dataloader.batch_size
    num_sentences = dataloader.dataset.max_sentences_per_article
    max_args_per_sentence = dataloader.dataset.max_args_per_sentence
    K = 15

    print("num_batches", len(dataloader))
    print("batch_size", batch_size)
    print("num_sentences", num_sentences)
    print("max_args_per_sentence", max_args_per_sentence)
    print("K", K)


    all_preds_span = []

    # Initialize usage lists for each label
    all_used_labels_p = []
    all_used_labels_a0 = []
    all_used_labels_a1 = []

    all_used_fx = []

    with torch.no_grad():
        # Wrap the dataloader with tqdm for batch progress
        for batch in tqdm(dataloader, desc="Processing Batches"):

            sentence_ids = batch['sentence_ids'].to(device)
            sentence_attention_masks = batch['sentence_attention_masks'].to(device)

            predicate_ids = batch['predicate_ids'].to(device)
            arg0_ids = batch['arg0_ids'].to(device)
            arg1_ids = batch['arg1_ids'].to(device)

            frameaxis_data = batch['frameaxis'].to(device)

            sentence_embeddings, predicate_embeddings, arg0_embeddings, arg1_embeddings = model.aggregation(sentence_ids, sentence_attention_masks, predicate_ids, arg0_ids, arg1_ids)

            # Process each span
            for sentence_idx in range(sentence_embeddings.size(1)):
                s_sentence_span = sentence_embeddings[:, sentence_idx, :]
                v_fx = frameaxis_data[:, sentence_idx, :]

                for span_idx in range(predicate_embeddings.size(2)):
                    v_p_span = predicate_embeddings[:, sentence_idx, span_idx, :]
                    v_a0_span = arg0_embeddings[:, sentence_idx, span_idx, :]
                    v_a1_span = arg1_embeddings[:, sentence_idx, span_idx, :]

                    mask_p = (v_p_span.abs().sum(dim=-1) != 0).float().bool()
                    mask_a0 = (v_a0_span.abs().sum(dim=-1) != 0).float().bool()
                    mask_a1 = (v_a1_span.abs().sum(dim=-1) != 0).float().bool()

                    #unsupervised.combined_autoencoder v_p, v_a0, v_a1, v_sentence, tau
                    output = model.unsupervised.combined_autoencoder(v_p_span,
                        v_a0_span,
                        v_a1_span,
                        mask_p,
                        mask_a0,
                        mask_a1,
                        s_sentence_span,
                        0.6)

                    all_used_labels_p.append(output["p"]["d"].cpu().numpy())
                    all_used_labels_a0.append(output["a0"]["d"].cpu().numpy())
                    all_used_labels_a1.append(output["a1"]["d"].cpu().numpy())

                mask_fx = (v_fx.abs().sum(dim=-1) != 0).float().bool()

                frameaxis_output = model.unsupervised_fx.frameaxis_autoencoder(v_fx, mask_fx, s_sentence_span, 0.6)

                all_used_fx.append(frameaxis_output["d"].cpu().numpy())


            # Forward pass
            _, span_logits, sentence_logits, combined_logits, _ = model(sentence_ids, sentence_attention_masks, predicate_ids, arg0_ids, arg1_ids, frameaxis_data, 0.5)
            combined_pred = (torch.softmax(combined_logits, dim=-1) > 0.5).float()

            all_preds_span.append(combined_pred.cpu().numpy())

            torch.cuda.empty_cache()

    predictions = np.vstack(all_preds_span)

    all_used_labels_p = np.vstack(all_used_labels_p)
    all_used_labels_a0 = np.vstack(all_used_labels_a0)
    all_used_labels_a1 = np.vstack(all_used_labels_a1)

    all_used_fx = np.vstack(all_used_fx)

    # reshape from (iterator (1), num sentences 24, num spans 10, batch size 64, classes 15) to (batch size 64, num sentences 24, num spans 10, classes 15)
    all_used_labels_p = all_used_labels_p.reshape(-1, num_sentences, max_args_per_sentence, K)
    all_used_labels_a0 = all_used_labels_a0.reshape(-1, num_sentences, max_args_per_sentence, K)
    all_used_labels_a1 = all_used_labels_a1.reshape(-1, num_sentences, max_args_per_sentence, K)

    all_used_fx = all_used_fx.reshape(-1, num_sentences, K)

    return predictions, all_used_labels_p, all_used_labels_a0, all_used_labels_a1, all_used_fx

In [ ]:
output = inspect(model, train_dataloader, device="cuda")

In [ ]:
predicted_labels, used_labels_p, used_labels_a0, used_labels_a1, used_fx = output

In [ ]:
category_lists_p = {category: [] for category in class_column_names}
category_lists_a1 = {category: [] for category in class_column_names}
category_lists_a0 = {category: [] for category in class_column_names}

category_lists_fx = {category: [] for category in class_column_names}

boundary = 0.4

elem_len = len(dataloader.dataset)
for elem_idx in range(elem_len):
    ds = dataloader.dataset[elem_idx]

    sent_len = len(ds["predicate_ids"])
    for sentence_idx in range(sent_len):
        span_len = len(ds["predicate_ids"][sentence_idx])
        for span_idx in range(span_len):

          for cat_idx, category in enumerate(class_column_names):
              if used_labels_p[elem_idx][sentence_idx][span_idx][cat_idx] > boundary:
                category_lists_p[category].append(ds["predicate_ids"][sentence_idx][span_idx].int().numpy())

              if used_labels_a0[elem_idx][sentence_idx][span_idx][cat_idx] > boundary:
                category_lists_a0[category].append(ds["arg0_ids"][sentence_idx][span_idx].int().numpy())

              if used_labels_a1[elem_idx][sentence_idx][span_idx][cat_idx] > boundary:
                category_lists_a1[category].append(ds["arg1_ids"][sentence_idx][span_idx].int().numpy())

        if used_fx[elem_idx][sentence_idx][cat_idx] > boundary:
          category_lists_fx[category].append(ds["frameaxis"][sentence_idx].float().numpy())

In [ ]:
category_lists_p

{'Capacity and Resources': [(array([ 211, 7536,    0,    0,    0,    0,    0,    0,    0,    0],
         dtype=int32),
   0.05981476),
  (array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32), 0.06666667),
  (array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32), 0.06666667),
  (array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32), 0.06666667),
  (array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32), 0.06666667),
  (array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32), 0.06666667),
  (array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32), 0.06666667),
  (array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32), 0.06666667),
  (array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32), 0.06666667),
  (array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32), 0.06666667),
  (array([1157,    0,    0,    0,    0,    0,    0,    0,    0,    0],
         dtype=int32),
   0.05981477),
  (array([962,   0,   0,   0,   0,   0,   0,   0,   0,   0], dtype=int32),
   0.05981476),
  (array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int3

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Ensure you have downloaded the necessary NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

def decode_tokens(token_dict, tokenizer, remove_stopwords=False, lemmatize=False):
    decoded_data = {}
    stop_words = set(stopwords.words('english')) if remove_stopwords else set()
    lemmatizer = WordNetLemmatizer() if lemmatize else None

    for category, token_lists in token_dict.items():
        decoded_data[category] = []
        for tokens in token_lists:
            if np.any(tokens > 0):
                # Convert tokens to a list if it's a tensor or numpy array
                if isinstance(tokens, torch.Tensor):
                    tokens = tokens.tolist()
                elif isinstance(tokens, np.ndarray):
                    tokens = tokens.tolist()

                # Decode the tokens
                decoded_text = tokenizer.decode(tokens, skip_special_tokens=True).strip()

                # Remove non-alphabetic characters (but keep spaces)
                decoded_text = re.sub(r'[^A-Za-z ]', '', decoded_text)

                # Tokenize, optionally lemmatize, and remove stop words
                words = word_tokenize(decoded_text)
                processed_words = [lemmatizer.lemmatize(word.lower()) if lemmatizer else word.lower() for word in words if word.lower() not in stop_words]

                # Join the words back into a string and ensure it's not empty
                processed_text = ' '.join(processed_words)
                if processed_text:
                    decoded_data[category].append(processed_text)

    return decoded_data

stop_words = set(stopwords.words('english'))

# Decode the token IDs for each ARG
decoded_predicate = decode_tokens(category_lists_p, tokenizer, remove_stopwords=True, lemmatize=True)
decoded_arg0 = decode_tokens(category_lists_a0, tokenizer, remove_stopwords=True, lemmatize=True)
decoded_arg1 = decode_tokens(category_lists_a1, tokenizer, remove_stopwords=True, lemmatize=True)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
